<a href="https://colab.research.google.com/github/chacha86/pythonai/blob/main/save_to_db.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pyspark

# 1. 추천 정보
df2 = spark.read.option("sep", ",").csv("/test/music/my_data.csv/").toDF('userid', 'artistid', 'playcount')

In [ ]:
%pyspark

df3 = df2.select(df2.userid.cast('int'), df2.artistid.cast('int'), df2.playcount.cast('int'))

In [ ]:
%pyspark
df4 = df3.filter(df3.userid.isNotNull() & df3.artistid.isNotNull() & df3.playcount.isNotNull())

In [ ]:
%pyspark
from pyspark.ml.recommendation import ALSModel
model = ALSModel.load('/test/model2')

In [ ]:
%pyspark
userRecs = model.recommendForAllUsers(3)

In [ ]:
%pyspark
from pyspark.sql.functions import *

tmp = userRecs.select(userRecs.userid, explode(userRecs.recommendations))
result = tmp.select(tmp.userid, tmp.col.artistid.alias('artistid'), tmp.col.rating.alias('rating'))

result

In [ ]:
%pyspark

result.write \
  .format("jdbc") \
  .option("url", "jdbc:mysql://ai-server-01:3306/t1") \
  .option("dbtable", "recommendation") \
  .option("user", "korea") \
  .option("password", "korea1234") \
  .mode("append") \
  .save()

In [ ]:
%pyspark
# 2. 유저 정보

## 로그인 아이디   -> id_유저번호
## 로그인 비밀번호 -> 1234
## 닉네임 -> name_유저번호
user_data = result.select('userid').distinct() \
.withColumn('loginid', concat(lit('id_'), result.userid)) \
.withColumn('loginpw', lit('1234')) \
.withColumn('nickname', concat(lit('name_'), result.userid)) \

In [ ]:
%pyspark
user_data.write \
  .format("jdbc") \
  .option("url", "jdbc:mysql://ai-server-01:3306/t1") \
  .option("dbtable", "user") \
  .option("user", "korea") \
  .option("password", "korea1234") \
  .mode("append") \
  .save()

In [ ]:
%pyspark
# 3. 아티스트 정보

artist = spark.read.option("sep", "\t").csv("/test/music/profiledata_06-May-2005/artist_data.txt").toDF('artistid', 'artistname')

In [ ]:
%pyspark
art2 = artist.filter((artist.artistid.isNotNull())  &  (artist.artistname.isNotNull())  )

In [ ]:
%pyspark

from pyspark.sql.functions import *
from pyspark.sql.types import *

# 문자 데이터가 포함된 행을 제거

## pandas의 map 방식

def my_is_digit(x) :
  if x.isdigit() :
    return True
  else :
    return False


my_digit = udf(my_is_digit, BooleanType())


art3 = art2.filter(my_digit(art2.artistid))

In [ ]:
%pyspark

art4 = art3.select(art3.artistid.cast("int"), art3.artistname)

In [ ]:
%pyspark
art4.write \
  .format("jdbc") \
  .option("url", "jdbc:mysql://ai-server-01:3306/t1") \
  .option("dbtable", "artist") \
  .option("user", "korea") \
  .option("password", "korea1234") \
  .mode("append") \
  .save()